In [83]:
import pandas as pd
import numpy as np
import requests
from get_mail_mailru import main as mail_main
from parser_broker_report import main as parser_main

In [84]:
# mail_main()
# parser_main()

In [85]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус', 'Номер сделки'])
transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)
transactions_executed = transactions[transactions['Статус'] == 'И']

In [86]:
transactions

,Дата заключения,Дата расчетов,Наименование,Код,Валюта,Вид,Количество,Цена,Сумма,НКД,Комиссия Брокера,Комиссия Биржи,Номер сделки,Статус
0,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.00,1633.00,0.0,0.98,0.15,3943134134,И
1,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.00,1633.00,0.0,0.98,0.15,3943134134,ЗО
2,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.00,1633.00,0.0,0.98,0.15,3943134134,О
3,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.00,5615.00,0.0,3.37,0.52,3943142919,ЗО
4,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.00,5615.00,0.0,3.37,0.52,3943142919,О
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,2024-03-28 19:04:54,2024-03-29,МосБиржа,MOEX,RUB,Покупка,30,219.00,6570.00,0.0,3.94,1.15,9972922899,ЗО
388,2024-03-29 11:38:01,2024-04-01,SBGDETF,SBGD,RUB,Покупка,84,19.91,1672.44,0.0,0.00,0.50,9987294084,ЗО
389,2024-03-29 11:38:01,2024-04-01,SBGDETF,SBGD,RUB,Покупка,16,19.92,318.72,0.0,0.00,0.09,9987294085,И
390,2024-03-29 11:38:01,2024-04-01,SBGDETF,SBGD,RUB,Покупка,16,19.92,318.72,0.0,0.00,0.09,9987294085,ЗО


In [87]:
ticker_list = list(transactions_executed['Код'].unique())
share_split_dict = {
    'FXGD': ['2022-02-17', 10],
    'SBMX': ['2021-06-09', 100],
    'FXUS': ['2022-01-24', 100],
    'FXRL': ['2021-11-24', 100],
    'FXRU': ['2022-02-17', 10],
    'FXDE': ['2021-12-15', 100]
}
replacement_dict = {
    'VTBE': 'RSHE',
    'RU000A102HB1': 'SU26227RMFS7'
}

In [88]:
def fix_split():
    ticker_list_copy = ticker_list.copy()
    for ticker in ticker_list_copy:
        if ticker == 'RU000A102HB1':
            new_ticker = 'SU26227RMFS7'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
        if ticker == 'VTBE':
            new_ticker = 'RSHE'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
    for ticker in ticker_list:
        if ticker in share_split_dict:
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] = \
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] * \
            share_split_dict[ticker][1]


fix_split()

In [89]:
def get_share_price_dict(tickers: list):
    share_price_dict = {}
    for ticker in tickers:
        if ticker in ['SBMX', 'AFKS']:
            round_numb = 3
        elif ticker in ['IRAO']:
            round_numb = 4
        else:
            round_numb = 2
        share = transactions_executed[transactions_executed['Код'] == ticker]
        share_price = ((share['Сумма'].sum() + share['Комиссия Брокера'].sum() + share['Комиссия Биржи'].sum()) / 
                            (share.loc[share['Вид'] == 'Покупка', 'Количество'].sum() - share.loc[share['Вид'] == 'Продажа', 'Количество'].sum())).round(round_numb)
        share_price_dict[ticker] = share_price
    return share_price_dict


share_price_dict = get_share_price_dict(ticker_list)

In [90]:
def get_share_amount_dict(tickers: list):
    share_amount_dict = {}
    for ticker in tickers:
        share_amount_dict[ticker] = transactions_executed.loc[transactions_executed['Код'] == ticker, 'Количество'].sum()
    return share_amount_dict


share_amount_dict = get_share_amount_dict(ticker_list)

In [91]:
def get_trading_dict():
    trading_dict = {}
    trading_mode_list = ['TQBR', 'TQTF', 'TQCB', 'TQOB']
    for id_trading in trading_mode_list:
        stocks = 'shares'
        if id_trading in ['TQOB', 'TQCB']:
            stocks = 'bonds'
        url = (f"https://iss.moex.com/iss/engines/stock/markets/{stocks}/boards/{id_trading}/"
            f"securities.csv?iss.meta=off&iss.only=marketdata&marketdata.columns=SECID,LAST")
        csv_text = requests.get(url).text.split('\n')
        trading_dict[id_trading] = csv_text
    return trading_dict


def get_last_prices_dict(tickers: list):
    last_prices_dict = {}
    trading_dict = get_trading_dict()
    for ticker in tickers:
        if ticker == 'RU000A102HB1':
            ticker =  'SU26227RMFS7'
        if ticker == 'VTBE':
            ticker = 'RSHE'
        for value in trading_dict.values():
            for line in value:
                line = line.split(';')
                if ticker in line:
                    last_prices_dict[ticker] = line[1]
    return last_prices_dict


last_prices_dict = get_last_prices_dict(ticker_list)

In [92]:
def get_commission_dict(tickers: list):
    commission_dict = {}
    for ticker in tickers:
        commission_dict[ticker] = (transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Брокера'] + \
                                     transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Биржи']) \
                                    .sum().round(2)
    return commission_dict


commission_dict = get_commission_dict(ticker_list)

In [93]:
def get_coupon_dict(tickers: list):
    coupon_dict = {}
    url = "https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQOB/securities.csv?iss.meta=off&iss.only" \
          "=securities&securities.columns=SECID,ACCRUEDINT "
    csv_text = requests.get(url).text.split('\n')

    for ticker in tickers:
        for line in csv_text:
            line = line.split(';')
            if ticker in line:
                coupon_dict[ticker] = line[1]
    return coupon_dict


coupon_dict = get_coupon_dict(ticker_list)

In [94]:
portfolio_dict = {
    'Котировки': last_prices_dict,
    'НКД': coupon_dict,
    'Количество': share_amount_dict,
    'Средняя цена': share_price_dict,
    'Комиссия': commission_dict
}

main = pd.DataFrame.from_dict(portfolio_dict)
main.index.name = 'Название'
main.loc[main['Котировки'] == '', 'Котировки'] = np.nan
main.dropna(axis=0, subset='Котировки', inplace=True)
main[['Котировки', 'НКД']] = main[['Котировки', 'НКД']].astype('float64')
# main.loc['SU25084RMFS3', 'Котировки'] = main.loc['SU25084RMFS3', 'Котировки'] * 10
# main.loc['SU25084RMFS3', 'Текущая цена'] = (main.loc['SU25084RMFS3', 'Котировки'] + main.loc['SU25084RMFS3', 'НКД']) * main.loc['SU25084RMFS3', 'Количество']
main['Текущая цена'] = main['Котировки'] * main['Количество']
main['P/L, руб.'] = main['Текущая цена'] - main['Средняя цена'] * main['Количество']
main['P/L, %'] = (main['Котировки'] * 100 / main['Средняя цена'] - 100).round(2)

# bond_price, bond_NKD, bond_amount, bond_average_price, bond_commission = \
#             main.loc['SU25084RMFS3', 'Котировки'], \
#             main.loc['SU25084RMFS3', 'НКД'], \
#             main.loc['SU25084RMFS3', 'Количество'], \
#             main.loc['SU25084RMFS3', 'Средняя цена'], \
#             main.loc['SU25084RMFS3', 'Комиссия']

# main.loc['SU25084RMFS3', 'Котировки'] = bond_price * 10
# main.loc['SU25084RMFS3', 'Текущая цена'] = (bond_price * 10 + bond_NKD) * bond_amount
# main.loc['SU25084RMFS3', 'P/L, руб.'] = main.loc['SU25084RMFS3', 'Текущая цена'] - bond_average_price * bond_amount
# main.loc['SU25084RMFS3', 'P/L, %'] = (((bond_price * 10 + bond_NKD) * 100 / bond_average_price) - 100).round(2)

main.to_csv('portfolio_main.csv', index_label=False)

In [95]:
df = pd.read_csv('portfolio_main.csv')
df.index.name = 'Название'

In [96]:
df

,Котировки,НКД,Количество,Средняя цена,Комиссия,Текущая цена,"P/L, руб.","P/L, %"
Название,,,,,,,,
SBMX,19.5710,NaN,1030,16.4160,4.98,20158.13,3249.65,19.22
SBER,299.9000,NaN,100,258.5800,20.75,29990.00,4132.00,15.98
MTSS,299.7500,NaN,60,279.3200,13.47,17985.00,1225.80,7.31
GAZP,157.7400,NaN,90,167.5900,13.57,14196.60,-886.50,-5.88
LKOH,7652.5000,NaN,3,6425.0000,13.00,22957.50,3682.50,19.11
MGNT,7823.0000,NaN,3,5769.7600,13.28,23469.00,6159.72,35.59
YNDX,3905.8000,NaN,2,2458.8700,3.74,7811.60,2893.86,58.85
CHMF,1863.0000,NaN,6,1405.0700,6.22,11178.00,2747.58,32.59
AFKS,23.6960,NaN,500,17.3780,7.35,11848.00,3159.00,36.36
